In [17]:
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

from langchain.sql_database import SQLDatabase
from langchain.agents.types import AgentType
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pyodbc 
import urllib  

In [18]:
# load envs

load_dotenv()

# Azure SQL DB
AZURE_SQL_SERVER_ADMIN = os.environ.get('AZURE_SQL_SERVER_ADMIN')
AZURE_SQL_SERVER_PWD = os.environ.get('AZURE_SQL_SERVER_PWD')
AZURE_SQL_SERVER = os.environ.get('AZURE_SQL_SERVER')
DRIVER = os.environ.get('DRIVER')

# OpenAI
AZURE_SQL_DATABASE = os.environ.get('AZURE_SQL_DATABASE')
AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.environ.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_VERSION = os.environ.get('AZURE_OPENAI_VERSION')
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get('AZURE_OPENAI_DEPLOYMENT_NAME')

In [19]:
# connect via pyodbc

CONNECTION_STRING = f'DRIVER={DRIVER};SERVER=tcp:{AZURE_SQL_SERVER},1433;DATABASE={AZURE_SQL_DATABASE};UID={AZURE_SQL_SERVER_ADMIN};PWD={AZURE_SQL_SERVER_PWD}'
cnxn = pyodbc.connect(CONNECTION_STRING)
cursor = cnxn.cursor()

In [20]:
# create engine

ODBC_CONNECTION_STRING = f"Driver={DRIVER};Server=tcp:{AZURE_SQL_SERVER},1433;Database={AZURE_SQL_DATABASE};Uid={AZURE_SQL_SERVER_ADMIN};Pwd={AZURE_SQL_SERVER_PWD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
ODBC_CONNECTION_STRING = urllib.parse.quote_plus(ODBC_CONNECTION_STRING) 
ODBC_CONNECTION_STRING = "mssql+pyodbc:///?odbc_connect=%s" % ODBC_CONNECTION_STRING

db_engine = create_engine(ODBC_CONNECTION_STRING)

# initialize db for RAG
db = SQLDatabase(db_engine)

In [21]:
oai = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
)

# test
message = HumanMessage(
    content="Translate this sentence from English to French. I love structured data yo!."
)
oai.invoke([message]).content

"J'adore les données structurées yo !"

In [22]:
# create sql toolkit for the agent
sql_toolkit = SQLDatabaseToolkit(db=db, llm=oai)

# create SQL agent
agent_executor = create_sql_agent(
    llm=oai,
    toolkit=sql_toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)


## Test

In [23]:
question = "How many fast moving items with minimum price per nos of over 1000 do we have?"

In [24]:
# run agent executor

output = agent_executor.invoke(question)
output['output']



> Entering new SQL Agent Executor chain...
To answer this question, I need to access the database and find the records that match the criteria. I assume that there is a table that contains information about the items, including their speed, price per nos, and quantity. But first, I need to confirm the tables in the database.
Action: sql_db_list_tables
Action Input: ""Medical Equipment Spare Part InventoriesThe database has one table which is "Medical Equipment Spare Part Inventories". I assume this table has the necessary information. I will need to look at the schema of this table to confirm if it has the fields I need to answer the question.
Action: sql_db_schema
Action Input: "Medical Equipment Spare Part Inventories"
CREATE TABLE [Medical Equipment Spare Part Inventories] (
	[Item_Code] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Item_Description] NVARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Part_No] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS 

'There are 17 fast moving items with a minimum price per nos of over 1000.'

## Retrieval Enhancements

In [25]:
# Step 1: Improve the quality of a system prompt

system_prompt = """ 

    You are the most powerful and skillfull expert in querying SQL Database to find answers to user's questions.

    Your main task is to answer the USER QUERY. Utilize CONTEXT and COLUMN DESCRIPTIONS for better query understanding.

    # CONTEXT
    Table 'Medical Equipment Spare Part Inventories' contains spare parts information for maintaining a spare parts inventory management program

    # COLUMN DESCRIPTIONS
    - Item Code: A code that uniquely identifies each type of item or part in the inventory.
    - Item Description: Provides a description of the item or part, which might include its general use or characteristics. This helps in identifying the item and understanding its application.
    - Part No: The part number, which is a unique identifier assigned to a part for the purposes of inventory and ordering.
    - Part Description: A brief description of the part, providing more details about its specifications or intended use in medical equipment.
    - Unit Of Measurement: The unit in which the item is measured.
    - Spare Part Type: Categorizes the part by its usage frequency or criticality, such as "Fast Moving Item" or "One Off".
    - Location: Indicates where the part is stored or located, which can be crucial for logistics and inventory retrieval. For example, a "Company Site Office" or a specific warehouse location.
    - Part Category: The category of the part, which helps in classifying parts into broader groups, such as "Biomedical" or "Electrical", aiding in management and reporting.
    - Is Expiry date Required: Specifies whether an expiry date is required for the part, which is important for parts that are sensitive to age or environmental conditions.
    - Min Nos: The minimum number of units that should be held in inventory for this part, which helps prevent understocking.
    - Max Nos: The maximum number of units that should be held in inventory, which helps in preventing overstocking and managing storage space efficiently.
    - Minimum Price Per Nos (RM): The minimum price per unit for the part, expressed in Malaysian Ringgit (RM). This could be used for budgeting and cost management.
    - Maximum Price Per Nos (RM): The maximum price per unit for the part, also in Malaysian Ringgit. This might reflect different supplier costs or quality grades.
    - Brand: The brand of the part, which can be a determinant of quality and preference for certain applications or compliance with specific equipment.
    - Status: Indicates the current status of the part in the inventory (Active, Obsolete or Out of Stock)
    - Expiry Age (In Month): If applicable, how many months until the part should be considered expired after receipt. This is particularly relevant for parts with critical performance characteristics that degrade over time.
    - Current Stock Level: The current quantity of the part available in stock.

    
    # MANDATORY REQUIREMENTS
    You never hallucinate.
    You always answer precisely and constuctively.

    # USER QUERY
    {query}

"""

In [26]:
question = "How many fast moving items do we have?"

output = agent_executor.invoke(system_prompt.format(query=question))
output['output']



> Entering new SQL Agent Executor chain...
To answer this question, I need to find the count of items from the 'Medical Equipment Spare Part Inventories' table where the 'Spare Part Type' is 'Fast Moving Item'. 

Action: sql_db_list_tables
Action Input: ""Medical Equipment Spare Part InventoriesThe 'Medical Equipment Spare Part Inventories' table exists. Now I will write an SQL query to count the number of items where 'Spare Part Type' is 'Fast Moving Item'. 

Action: sql_db_query_checker
Action Input: "SELECT COUNT(*) FROM `Medical Equipment Spare Part Inventories` WHERE `Spare Part Type` = 'Fast Moving Item'"SELECT COUNT(*) FROM `Medical Equipment Spare Part Inventories` WHERE `Spare Part Type` = 'Fast Moving Item'The SQL query is correct. Now I will execute it to get the answer.

Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM `Medical Equipment Spare Part Inventories` WHERE `Spare Part Type` = 'Fast Moving Item'"Error: (pyodbc.ProgrammingError) ('42000', "[42000] [Micros

'There are 235 fast moving items.'